<a href="https://colab.research.google.com/github/nooneatall533/cse6040/blob/main/2022-09-08--nb1-spdot-defaultdict-nb4teaser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nb1+2 topics #

Topics in this note:

* Exercise nb1.1.3 — contrasting three methods
* Extension: Sparse dot products
* Default dictionaries
* Notebook 4 sneak-peak (Google Colab)

## nb1.1.3 — Decompressing a sparse vector ##

Consider a sparse vector as defined for problem nb1.1.3. A logical vector of length `n`, which is expected to have many zero elements, is stored using a pair of lists that record only the nonzero values and their indices. These two lists are, in turn, wrapped into a dictionary.

In [1]:
d = {}
d['inds'] = [0,   3,   7,   3,   3,   5, 1]
d['vals'] = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

This corresponds to a logical (mathematical) vector, `x_true`, whose elements are:

In [2]:
x_true = [1.0, 7.0, 0.0, 11.0, 0.0, 6.0, 0.0, 3.0]

Thus, the object `d` is a compact representation of `x_true` when there are many zero components.

Recall that the task in nb1.1.3 is to decompress `d`, producing `x_true`. A natural method is to construct each element of the output, one at a time. For each output, the method scans the sparse vector and accumulates all matching entries.

In [3]:
def decomp_i(i, d):
    x_i = 0.0
    for k, v in zip(d['inds'], d['vals']): # O(m)
        if k == i:
            x_i += v
    return x_i

In [4]:
def decomp_i__alt(i, d):
    vals_i = [v for k, v in zip(d['inds'], d['vals']) if k == i]
    return sum(vals_i)

In [8]:
def decomp__v0(d, n=None):
    # Input is of length `m`
    # Output is of length `n`
    # If possible, we'd like a method that scales like `O(m + n)`

    if n is None:
        n = max(d['inds']) + 1

    x = []
    x = [decomp_i(i,d) for i in range(n)]
    #for i in range(n): # O(n)
        # build element x[i]
        #x.append(decomp_i(i, d))

    return x

x = decomp__v0(d)
print(x)
assert x == x_true

[1.0, 7.0, 0.0, 11.0, 0.0, 6.0, 0.0, 3.0, 1.0, 7.0, 0.0, 11.0, 0.0, 6.0, 0.0, 3.0]


AssertionError: 

The problem with the above is that it reads `d` for _every_ output element. Can we accomplish the same thing touching each input only once (or only a few times)? Yes!

In [ ]:
def decomp__v1(d, n=None):
    if n is None:
        n = max(d['inds']) + 1
    y = [0] * n  # O(n)
    for k, v in zip(d['inds'], d['vals']): # O(m)
        y[k] += v
    return y

y = decomp__v1(d)
print(y)
assert y == x_true

[1.0, 7.0, 0, 11.0, 0, 6.0, 0, 3.0]


## Follow-up exercise: Sparse dot products ##

Consider a dense dot product where the vector elements (or components) include zeros:

In [11]:
x = [17, 0, 0, -20, -20, 14, 0, 0, -6, 0]
y = [-6, 0, 13, 0, 2, 0, 0, 7, -2, 0]

def dot(x, y):
    return sum(ex*ey for ex, ey in zip(x, y))

dot(x, y)

-130

Here is a variation on this code designed to resemble more closely the mathematical formula for a dot product, which iterates over an index:

$$\mbox{dot}(x, y) = \sum_{i=0}^{n-1} x_i y_i.$$

In [12]:
def dot__v2(x, y):
    assert len(x) == len(y)
    return sum(x[i]*y[i] for i in range(len(x)))

dot__v2(x, y)

-130

Suppose we want to exploit the zeros by avoiding storing and operating on them. Here are two sparse vectors in the style of `nb1.1.3`:

In [9]:
dx = {'inds': [0, 3, 4, 5, 8], 'vals': [17, -20, -20, 14, -6]}
dy = {'inds': [0, 2, 4, 7, 8], 'vals': [-6, 13, 2, 7, -2]}

**Question:** What does a dot product look like for this sparse representation?

The terms of the dot product (i.e., $x_i y_i$) are nonzero only when _both_ $x_i \neq 0$ and $y_i \neq 0$. We can detect the common $i$ indices between the two sparse vectors by converting the indices to sets and then finding their intersection:

In [29]:
def common_inds(dx, dy):
    inds_x = set(dx['inds'])
    inds_y = set(dy['inds'])
    return inds_x & inds_y

common_inds(dx, dy)

def sparse_dot_mine(dx,dy):
  commoninds = [i for i in common_inds(dx,dy)]
  res = 0
  xvector = {k:v for k,v in zip(dx['inds'], dx['vals'])}
  yvector = {k:v for k,v in zip(dy['inds'], dy['vals'])}
  res = sum(xvector[commoninds[i]] * yvector[commoninds[i]] for i in range(len(commoninds)))
  return res

%timeit sparse_dot_mine(dx,dy)

3.09 µs ± 67.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


From this building block, here's a first cut at a dot product "derived" from `dot__v2`:

In [30]:
def sp_dot__v0(dx, dy):
    dot = 0
    for i in common_inds(dx, dy):    # O(m) where m == max nnz per vector
        loc_x = dx['inds'].index(i)  # Also O(m) because of the index-search
        loc_y = dy['inds'].index(i)
        x_i = dx['vals'][loc_x]
        y_i = dy['vals'][loc_y]
        dot += x_i * y_i
    return dot

%timeit sp_dot__v0(dx, dy)

1.77 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


If each vector has an average of $m$ nonzeros, then the cost of this procedure is $O(m^2)$, since there is a hidden $O(m)$ cost to search the list to find the index location.

**Reducing the cost.** To reduce the cost of search, we can ask whether there is some preprocessing that might help. Indeed, in a single pass over the sparse vector, we can construct a "lookup table" that maps indices to values.

In [ ]:
def gen_lookup(d): # `d` is a sparse vector
    t = {} # Lookup table to map an index to its value
    for i, v in zip(d['inds'], d['vals']):
        if i not in t:
            t[i] = 0
        t[i] += v
    return t

gen_lookup(dx) # Demo

{0: 17, 3: -20, 4: -20, 5: 14, 8: -6}

> _Note:_ This code will work even in the case of repeated indices, even though our example does not include that case.

In [ ]:
print(d)
print(gen_lookup(d))
print(x_true)

{'inds': [0, 3, 7, 3, 3, 5, 1], 'vals': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]}
{0: 1.0, 3: 11.0, 7: 3.0, 5: 6.0, 1: 7.0}
[1.0, 7.0, 0.0, 11.0, 0.0, 6.0, 0.0, 3.0]


Combining these two building blocks, here is a more efficient sparse dot:

In [ ]:
def sp_dot__v1(dx, dy):
    tx = gen_lookup(dx)
    ty = gen_lookup(dy)
    I = set(tx.keys()) & set(ty.keys()) # common indices
    return sum(tx[i]*ty[i] for i in I)

sp_dot__v1(dx, dy)

-130

**Benchmarking.** Let's compare the speed of these two sparse dot product implementations against the dense case. Let's start with a small example.

In [31]:
def decomp(d, n_max=None):
    if n_max is None:
        n_max = (max(d['inds']) if d['inds'] else 0) + 1
    y = [0] * n_max
    for i, v in zip(d['inds'], d['vals']):
        y[i] += v
    return y

print("* Dense case...")
x = decomp(dx)
y = decomp(dy)
%timeit dot(x, y) # Could also try `dot__v2`

print("\n* Sparse dot, version 0...")
%timeit sp_dot__v0(dx, dy)

print("\n* Sparse dot, version 1...")
%timeit sp_dot__v1(dx, dy)

* Dense case...
1.1 µs ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

* Sparse dot, version 0...
1.76 µs ± 364 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

* Sparse dot, version 1...


NameError: name 'sp_dot__v1' is not defined

On small inputs, you can see the exact _opposite_ of what you might expect! That's because conversion to sets and construction or manipulation of lookup tables carries an overhead that outweights the benefits of sparse storage when the vectors are small.

Instead, let's try large inputs. Here is a helper function to generate a large random sparse input vector:

In [ ]:
def gen_rand_sparse_vector(n=10, nnz=10, max_val=20, unique=True):
    from random import randint, sample, random, shuffle
    nnz_unique = nnz
    vals_unique = [(i, randint(-max_val, max_val)) for i in sample(range(n), nnz)]
    d_true = {'inds': [i for i, _ in sorted(vals_unique)],
              'vals': [v for _, v in sorted(vals_unique)]}
    if unique:
        return d_true

    vals = []
    for i, v in vals_unique:
        if random() < 0.2: # Duplicate!
            n_dups = randint(2, 5)
            vals += [(i, w) for w in partition_value(v, n_dups)]
        else:
            vals += [(i, v)]
    shuffle(vals)
    d_input = {'inds': [i for i, _ in vals],
               'vals': [v for _, v in vals]}
    return d_input

Let's use `gen_rand_sparse_vector` to generate random sparse input vectors of length 100,000 elements with 1,000 nonzeros each.

In [ ]:
N = 100_000
dx_big = gen_rand_sparse_vector(n=N, nnz=1_000)
dy_big = gen_rand_sparse_vector(n=N, nnz=1_000)

print("* dx_big:", len(dx_big['inds']), "nonzeros.")
print("* dy_big:", len(dx_big['inds']), "nonzeros.")
print("* common indices:", len(common_inds(dx_big, dy_big)), "nonzeros.")

* dx_big: 1000 nonzeros.
* dy_big: 1000 nonzeros.
* common indices: 11 nonzeros.


Let's try first converting to dense vectors:

In [ ]:
x_big = decomp(dx_big, n_max=N)
y_big = decomp(dy_big, n_max=N)
print(dot(x_big, y_big)) # note the "correct" answer
%timeit dot(x_big, y_big)

-449
6.53 ms ± 59.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now let's try our two sparse methods:

In [ ]:
print(sp_dot__v0(dx_big, dy_big)) # check answer against the dense case
%timeit sp_dot__v0(dx_big, dy_big)

print(sp_dot__v1(dx_big, dy_big)) # check answer
%timeit sp_dot__v1(dx_big, dy_big)

-449
194 µs ± 3.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-449
451 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


You should observe these to be much faster now.

## Default dictionaries ##

The `gen_lookup` function contained an instance of a common pattern with dictionaries:

```python
    t = {}
    for i in ...:
        if i not in t:
            t[i] = 0
        t[i] += ...
```

Before accumulating `t[i]`, the code verifies that the key `i` exists. If it does not, it first creates an "empty" entry, initialized to zero, and then does the accumulate.

**Example:** Suppose we wish to count the number of occurrences of a character in a string.

In [33]:
s = "bbbaaaabaaa"

In this case, `'a'` occurs 7 times and `'b'` occurs 4 times. Let's say we want to construct a dictionary `count` such that `count['a'] == 7` and `count['b'] == 4`.

The following code does _not_ work! Try uncommenting it to see:

In [ ]:
#count = {}
#for c in s:
#    count[c] += 1
#count

Instead, we need something like the following:

In [34]:
count = {}
for c in s:
    if c not in count:
        count[c] = 0
    assert c in count
    count[c] += 1
count

{'b': 4, 'a': 7}

Default dictionaries give us a way to simplify this code:

In [37]:
from collections import defaultdict
count = defaultdict(int)

for c in s:
    count[c] += 1
count

defaultdict(int, {'b': 4, 'a': 7})

The `defaultdict(...)` constructor is another example of a higher-order function: its single argument is a _function_. The function must have the property that when it is called with no inputs it produces a value as its output, where the value may be considered an initial value for nonexistent keys.

For instance, recall:

In [ ]:
int()

0

Therefore, the `defaultdict(int)` object will use `int()` whenever it needs a new initial value.

> The other basic built-in Python objects have a similar property. Try `float()`, `str()`, `list()`, `set()`, and even `dict()`.

A major pitfall with default dictionaries is that even just referencing a key causes it to be created. Example:

In [ ]:
print(count)
count['abc']  # Not doing anything here — not assigning, not using
print(count)

defaultdict(<class 'int'>, {'b': 4, 'a': 7})
defaultdict(<class 'int'>, {'b': 4, 'a': 7, 'abc': 0})


That can lead to blow-ups in storage (and time!). So, do be careful not to reference keys unnecessarily.

**An alternative: `dict.get`.** Default dictionaries aren't the only way. Recall that if `d` is a dictionary, then `d.get(key, default_value)` will return `default_value` if `key` does not exist in `d`:

In [ ]:
'x' in count, count.get('x', 0), count.get('a', 0)

(False, 0, 7)

Thus:

In [36]:
count = {}
for c in s:
    count[c] = count.get(c, 0) + 1
count

{'b': 4, 'a': 7}

**Exercise:** What does this code produce?

In [38]:
def default_value():
    return -20

count2 = defaultdict(default_value)
for c in s:
    count2[c] += 1
count2

defaultdict(<function __main__.default_value()>, {'b': -16, 'a': -13})

**Aside: Another alternative, `Counter` objects.** The `collections` module implements many useful objects and functions. One is `Counter`, which does exactly what we need in our letter-counting problem.

In [ ]:
from collections import Counter
Counter(s)

Counter({'b': 4, 'a': 7})

In [ ]:
isinstance(Counter(s), dict)

True

Although `Counter` constructs a special object of that type, in fact, it is derived from a dictionary so it can be used as such.

## Teaser for Notebook 4 ##

See this [Google Colab notebook](https://colab.research.google.com/drive/1-MlOoW5y2TznOm_LmBjlArjbkwvMrykJ?usp=sharing) and observe how two computations that should produce identical answers appear to be ever so slightly different. The question we closed with is, are you okay with that? How would you know?

## Summary ##

1. From nb1.1.3, remember your analytical goal, which is to strive for linear time methods.
2. In the sparse dot product problem, a judicious combination of the right data structures helped to achieve efficient code, at least for large-enough inputs.
3. Default dictionaries help address a common pattern with dictionaries, which is creating a key with a default value when the key does not exist. There are alternatives, too, so keep them in mind.
4. The next homework (Notebook 4) will be about floating-point arithmetic. It will help answer some fundamental questions, namely, how do we represent real numbers in a finite (i.e., efficient) way on a computer, and how do we reason about the correctness of programs that manipulate such numbers?